Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name, email and UFID.
Please do not modify instruction cells or any cells with automated tests (marked with `[ASSERTS]`). Note: you can add new cells if you need them, but answers must be in the cells with `YOUR CODE HERE` or "YOUR ANSWER HERE" comments.

---

## Homework 2: Regression and Ensembles

## Preamble: Write your Name, Email and UFID

In [1]:
NAME = 'Tikaharu Sharma'
EMAIL = 'tikaharusharma@ufl.edu'
UFID = 73535666

# YOUR CODE HERE

print('Homework 2 -- name: {}, email: {}, UFID: {}\n'.format(NAME, EMAIL, UFID))

Homework 2 -- name: Tikaharu Sharma, email: tikaharusharma@ufl.edu, UFID: 73535666



In [2]:
""" [ASSERTS] Check that your name, email, and UFID is filled in."""
assert NAME != '' and NAME != 'Your name here.' and len(NAME) > 3
assert EMAIL != '' and EMAIL != 'Your email here.' and len(EMAIL) > 7
assert type(UFID) == int and UFID != 12345678 and UFID >= 10000000 and UFID <= 99999999

## Academic Integrity

### <span style="color:red;">This is an individual assignment. Academic integrity violations (i.e., cheating, plagiarism) will be reported to SCCR!</span><br/>
#### The official CISE policy recommended for such offenses is a course grade of E. Additional sanctions may be imposed by SCCR such as marks on your permanent educational transcripts, dismissal or expulsion.
#### Reminder of the Honor Pledge: On all work submitted for credit by Students at the University of Florida, the following pledge is either required or implied: *"On my honor, I have neither given nor received unauthorized aid in doing this assignment."*

#### Acknowledgement: Do you acknowledge and understand the academic integrity warning above? 

In [3]:
academic_integrity_acknowledgement = True
# YOUR CODE HERE


In [4]:
""" [ASSERTS] Check that you acknowledge the academic integrity warning, you understand it and have been reminded of the UF Honor Pledge."""
assert academic_integrity_acknowledgement

---
# [Task 1] (20 points) Loading and preprocessing.

### [Task 1] We will use the Bike Sharing dataset (hourly). A version of this dataset is included in the homework handout archive.
### This dataset contains features of users bike sharing/rental on an hourly basis.
### The task is to predict how many users are sharing/renting a bike.
### You will load the data and preprocess it.

#### The following cell's code (import statements etc.) is provided for you and you should not need to change it.

In [5]:
# Load packages we need
import sys
import os
import time

import numpy as np
import pandas as pd
import sklearn

from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 16})

# Let's check our software versions
print('------------')
print('### Python version: ' + __import__('sys').version)
print('### NumPy version: ' + np.__version__)
print('### Scikit-learn version: ' + sklearn.__version__)
print('------------')

def var_exists(var_name):
    return (var_name in globals() or var_name in locals())

------------
### Python version: 3.10.19 (main, Oct 21 2025, 16:37:10) [Clang 20.1.8 ]
### NumPy version: 2.2.5
### Scikit-learn version: 1.7.2
------------


#### This is the seed we will use, do not change it.

In [6]:
# set the seed
seed = 42
np.random.seed(seed)

prop_vec = [16, 2, 2] # proportions for train - val - test splits

In [7]:
""" [ASSERTS] Check seed. """
assert seed == 42

### Loading data (set the path correctly so it runs on your machine --- don't submit the data file with your notebook).
#### Note: this dataset has missing values (artificially introduced), which you'll need to fill in before you can train a model

In [8]:
"""Fill in the path to the directory where 'bikesharehour.csv.gz' is located.
"""
data_root = 'data' #put the path here
# YOUR CODE HERE


In [9]:

dataset_fp = os.path.join(data_root, 'bikesharehour.csv.gz')
assert os.path.exists(dataset_fp), f"Dataset not found ({dataset_fp})!"
df = pd.read_csv(dataset_fp, compression='gzip', header=0, na_values='?')

# Check that we loaded the data as expected
df_expected_shape = (17379, 15)
assert df.shape == df_expected_shape, 'Unexpected shape of df!'

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      16320 non-null  float64
 1   year        16231 non-null  float64
 2   month       16304 non-null  float64
 3   hour        16254 non-null  float64
 4   holiday     16276 non-null  float64
 5   weekday     16282 non-null  float64
 6   workingday  16297 non-null  float64
 7   weathersit  16324 non-null  float64
 8   temp        16242 non-null  float64
 9   atemp       16271 non-null  float64
 10  hum         16252 non-null  float64
 11  windspeed   16281 non-null  float64
 12  registered  16244 non-null  float64
 13  nsqrtc      16261 non-null  float64
 14  count       17379 non-null  int64  
dtypes: float64(14), int64(1)
memory usage: 2.0 MB


## [Task 1a] (5 points) Show the first 9 rows of the dataframe df. 

In [10]:
"""Put your code here.
"""
## what does the data look like?
# YOUR CODE HERE
df.head(9)

,season,year,month,hour,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,registered,nsqrtc,count
0,1.0,0.0,NaN,0.0,0.0,6.0,0.0,1.0,NaN,0.0,0.0,0.0,14.0,-51.0,16
1,1.0,0.0,NaN,1.0,0.0,6.0,0.0,1.0,NaN,0.0,0.0,0.0,32.0,-8.0,40
2,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,27.0,-71.0,32
3,1.0,0.0,1.0,3.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,10.0,-5.0,13
4,1.0,0.0,1.0,4.0,NaN,6.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1
5,1.0,0.0,1.0,5.0,0.0,6.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,1
6,1.0,0.0,1.0,6.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,2
7,1.0,0.0,1.0,7.0,0.0,6.0,0.0,1.0,0.0,NaN,0.0,0.0,2.0,-3.0,3
8,1.0,0.0,1.0,8.0,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0,3.0,8


In [11]:
# grab all the data as a numpy matrix
all_xy = df.to_numpy()

col_names = [c for c in df.columns]
features = col_names[:-1]
target = col_names[-1]

# what are the features and what is the target?
print(f"features: {features}")
print(f"target: {target}")


features: ['season', 'year', 'month', 'hour', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'registered', 'nsqrtc']
target: count


## [Task 1b] (5 points) Answer the following questions by setting variables (hardcoding or computing the answer) from 'all_xy'. 
## (1) If we do supervised learning to predict 'count' based on our features is this classification or regression? (Set the corresponding variable to True.) 
## (2) How many NaNs are there in columns 'holiday','temp', and 'count'?

In [12]:
"""Put your code here and set the variables appropriately."""

classification = False
regression = True

# Compute NaN counts from the dataframe `df` (available in the notebook scope)
holiday_NaNs = int(df['holiday'].isna().sum())
temp_NaNs = int(df['temp'].isna().sum())
count_NaNs = int(df['count'].isna().sum())


In [13]:
""" [ASSERTS] Check 1b completed. """

assert var_exists('classification') and var_exists('regression') and classification != regression
assert var_exists('holiday_NaNs') and holiday_NaNs >= 0
assert var_exists('temp_NaNs') and temp_NaNs >= 0
assert var_exists('count_NaNs') and count_NaNs >= 0


### [Task 1c] (5 points) Let's impute the missing values! Use Scikit-learn's SimpleImputer to replace all NaNs in 'all_xy' with the *most frequent* value in each column. Use copy=True and store the results in 'all_xy_noNaNs' 

In [14]:
"""Put your code here (~2-3 lines).
"""
from sklearn.impute import SimpleImputer
# YOUR CODE HERE
imputer = SimpleImputer(strategy="most_frequent", copy=True)
all_xy_noNaNs = imputer.fit_transform(all_xy)


In [15]:
""" [ASSERTS] Check 1c completed. """

assert var_exists('all_xy_noNaNs') and all_xy_noNaNs.shape == df_expected_shape


### [Task 1d] (5 points) Min-max normalize the features (to [0,1] range) using sklearn's MinMaxScaler. Store the results into 'scaled_all_x'. Then split the data into train, val, test according to the proportion in prop_vec using sklearn's train_test split. Store the results into 'train_x', 'train_y', 'test_x', 'test_y', 'val_x', 'val_y'. 

In [16]:
"""Put your code here (~5-7 lines).
"""

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# YOUR CODE HERE
scaler = MinMaxScaler(feature_range=(0, 1))
all_x = all_xy_noNaNs[:, :-1]
all_y = all_xy_noNaNs[:, -1]
scaled_all_x = scaler.fit_transform(all_x)
train_x, temp_x, train_y, temp_y = train_test_split(scaled_all_x, all_y, test_size=0.2, random_state=seed)
val_x, test_x, val_y, test_y = train_test_split(temp_x, temp_y, test_size=0.5, random_state=seed)

""" [ASSERTS] Check 1d completed. """

' [ASSERTS] Check 1d completed. '

In [17]:
""" [ASSERTS] Check split for task 1d. """
assert var_exists('scaled_all_x')
assert var_exists('train_x') and var_exists('train_y') and train_x.shape[0] == train_y.shape[0]
assert var_exists('val_x') and var_exists('val_y') and val_x.shape[0] == val_y.shape[0]
assert var_exists('test_x') and var_exists('test_y') and test_x.shape[0] == test_y.shape[0]
assert train_x.shape == (13903, 14) and val_x.shape == (1738, 14) and test_x.shape == (1738, 14)
assert np.amin(train_x) >= 0.0 and np.amax(train_x) <= 1.0


## [Task 2] (30 points) Evaluating and training linear models.

### [Task 2a] (5 points) Fill in the code to calculate and return the metrics.

In [18]:
"""Fill in your code below (~5-7 lines).
"""
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def r2_mse_mae_eval(model, tr_x, tr_y, v_x, v_y, pref='', verb=True):

    """Fill in your code below (~1-2 lines).
    """
    # R^2 the coefficient of determination
    # YOUR CODE HERE
    train_r2 = model.score(tr_x, tr_y)
    val_r2 = model.score(v_x, v_y)
    
    if verb:
        print(f"{pref}Train R^2: {train_r2:.3f}, Val  R^2: {val_r2:.3f}")

    train_pred = model.predict(tr_x)
    val_pred = model.predict(v_x)

    """Fill in your code below (~1-2 lines).
    """
    # YOUR CODE HERE
    train_mse = mean_squared_error(train_pred, tr_y)
    val_mse = mean_squared_error(val_pred, v_y)

    if verb:
        print(f"{pref}Train MSE: {train_mse:.3f}, Val MSE: {val_mse:.3f}")

    """Fill in your code below (~1-2 lines).
    """
    # YOUR CODE HERE
    train_mae = mean_absolute_error(train_pred, tr_y)
    val_mae = mean_absolute_error(val_pred, v_y)

    if verb:
        print(f"{pref}Train MAE: {train_mae:.3f}, Val MAE: {val_mae:.3f}")

    return train_r2, val_r2, train_mse, val_mse, train_mae, val_mae

In [19]:
""" [ASSERTS] Check split for task 2a. """
assert var_exists('r2_mse_mae_eval')


### [Task 2b] (5 points) Train a linear regression model using the default (hyper)parameters. Call the resulting trained model 'lrmodel'.

In [20]:
"""Put your code here (~1-2 lines).
"""
from sklearn.linear_model import LinearRegression, Ridge

# YOUR CODE HERE
lrmodel = LinearRegression()
lrmodel.fit(train_x, train_y)

_ = r2_mse_mae_eval(lrmodel, train_x, train_y, val_x, val_y, pref='[{}] '.format(lrmodel.__class__.__name__))

[LinearRegression] Train R^2: 0.841, Val  R^2: 0.845
[LinearRegression] Train MSE: 5268.343, Val MSE: 5184.028
[LinearRegression] Train MAE: 39.637, Val MAE: 38.503


### [Task 2b] How good is that model? (A few sentences is fine.)

In [21]:
""" Remove the 'raise NotImplementedError' line. Write your answer as a comment in the place provided.  (Do not change the cell type from code to markdown.)"""
# 
## Answer: 
#
# YOUR CODE HERE
# The linear regression model performs reasonably well: validation R^2 is high and close to training R^2,
# and validation MSE/MAE are only slightly worse than training, which suggests limited overfitting
# and decent generalization. It is not perfect, though, so some nonlinear structure likely remains unmodeled.


" Remove the 'raise NotImplementedError' line. Write your answer as a comment in the place provided.  (Do not change the cell type from code to markdown.)"

### [Task 2c] (5 points) Fill in the code below to setup a grid search. Concatenate the train and val sets into 'search_x' and 'search_y'.

In [22]:
## some code to do a grid search and automatically train & evaluate the model with the best hyperparams.
from sklearn.model_selection import GridSearchCV

def do_grid_search(model, param_grid, x, y):
    gs = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error')
    gs_res = gs.fit(x, y)
    return  gs_res.best_params_


def search_train_eval(model, param_grid, tr_x=train_x, tr_y=train_y, v_x=val_x, v_y=val_y):

    """Put your code here (~2-3 lines). Concatenate the train and val sets into 'search_x' and 'search_y'.
    """
    # YOUR CODE HERE
    search_x = np.concatenate((tr_x, v_x), axis=0)
    search_y = np.concatenate((tr_y, v_y), axis=0)
    
    hyperparams = do_grid_search(model, param_grid, search_x, search_y)
    
    class_obj = type(model)
    m = class_obj(**hyperparams).fit(tr_x, tr_y)
    
    cn = str(class_obj).split("'")[1]
    cn = cn.split('.')[-1]
    print(f"{cn}({hyperparams})")

    r2_mse_mae_eval(m, tr_x, tr_y, v_x, v_y, pref='\t')

    return m

In [23]:
""" [ASSERTS] Check task 2c. """
assert var_exists('search_train_eval')


### [Task 2d] (10 points) Take a look at the code of search_train_eval() and do_grid_search(). Answer the following questions: 
### 1. Can you do LAD regression with sklearn? Why or why not? (Justify your answer.) Hint: take a look at the sklearn documentation and the course slides.
### 2. Why is the scoring function for the grid search 'neg_mean_squared_error' (as opposed to 'mean_squared_error')? 
### 3. Why is it okay to do the search over search_x and search_y which are the concatenation of the train and 'validation sets?

In [24]:
""" Remove the 'raise NotImplementedError' line. Write your answer as a comment in the place provided.  (Do not change the cell type from code to markdown.)"""
# 
### Hint: take a look at the documentation of scikit-learn and think.
## Answer: 
# 1. We can do LAD regression with sklearn but not with LinearRegression or Ridge. LinearRegression optimize L2
# loss, not L1 absolute loss. The most direct way to achieve LAD regression within scikit-learn is by using the 
# QuantileRegressor class and setting the quantile parameter to 0.5. 
# 
# 2. GridSearchCV always maximizes its scoring metric. MSE should be minimized, so sklearn
# uses 'neg_mean_squared_error' so "larger is better" still works (less MSE -> less negative -> higher score).
#
# 3. GridSearchCV does its own internal cross-validation splits on the data, so it is okay to search over 
# search_x and search_y.
# YOUR CODE HERE


" Remove the 'raise NotImplementedError' line. Write your answer as a comment in the place provided.  (Do not change the cell type from code to markdown.)"

### [Task 2e] (5 points) Write the code below to do a grid search on a LassoLars model. Call the resulting model 'lassolars_model'. Be sure to tune the regularization constant and 'fit_intercept'. Make sure training converges and that you set the seed.

In [25]:
"""Put your code here (~2-3 lines).
"""

# YOUR CODE HERE
from sklearn.linear_model import LassoLars

param_grid = {'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1.0], 'fit_intercept': [True, False]}
lassolars_model = search_train_eval(LassoLars(max_iter=5000, random_state=seed), param_grid)


LassoLars({'alpha': 0.01, 'fit_intercept': True})
	Train R^2: 0.841, Val  R^2: 0.845
	Train MSE: 5268.786, Val MSE: 5184.698
	Train MAE: 39.630, Val MAE: 38.497


In [26]:
""" [ASSERTS] Check task 2e. """
assert var_exists('lassolars_model')


## [Task 3] (25 points) Let's train polynomial regression models!

### [Task 3a] (15 points) Use PolynomialFeatures to create a version of the data with all features of degree 3. Follow the provided instructions.

In [27]:
"""Put your code here (~10-15 lines).
    1. Use PolynomialFeatures to create a version of the data with all features of degree 3. Make sure to allow interactions (interaction_only=False) and set include_bias=False.
    Store the result in 'all_x_pf'. Ensure that you make a copy of the original data and you use the scaled features ('scaled_all_x')!
    2. Split the data ('all_x_pf') into train-val-test using proportion from 'prop_vec' and save the result as 'train_{x,y}_pf', 'val_{x,y}_pf', and 'test_{x,y}_pf'.
    3. Train three models for comparison. 
        (a) The first is a ridge regression model 'ridge' on the original scaled data (no polynomial features) where you tuned the hyperparameters.
        (b) The second is a linear regression model 'lr_pf' on the polynomial features data.
        (c) The third is a regularized "linear" model (ridge regression) on the polynomial features data where you tuned the hyperparameters (including regularization constant ensuring alpha >= 1.0).
"""
# YOUR CODE HERE
from sklearn.preprocessing import PolynomialFeatures

polyf = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
all_x_pf = polyf.fit_transform(scaled_all_x.copy())

train_x_pf, temp_x_pf, train_y_pf, temp_y_pf = train_test_split(
    all_x_pf, all_y, test_size=(prop_vec[1] + prop_vec[2]) / sum(prop_vec), random_state=seed
)
val_x_pf, test_x_pf, val_y_pf, test_y_pf = train_test_split(
    temp_x_pf, temp_y_pf, test_size=prop_vec[2] / (prop_vec[1] + prop_vec[2]), random_state=seed
)

ridge_grid = {"alpha": [0.01, 0.1, 1.0, 10.0, 100.0], "fit_intercept": [True, False]}
ridge_nopf = search_train_eval(Ridge(random_state=seed), ridge_grid)

lr_pf = LinearRegression().fit(train_x_pf, train_y_pf)
_ = r2_mse_mae_eval(lr_pf, train_x_pf, train_y_pf, val_x_pf, val_y_pf, pref='[LinearRegression PF] ')

ridge_pf_grid = {"alpha": [1.0, 10.0, 100.0, 1000.0], "fit_intercept": [True, False]}
ridge_pf = search_train_eval(Ridge(random_state=seed), ridge_pf_grid, train_x_pf, train_y_pf, val_x_pf, val_y_pf)

Ridge({'alpha': 1.0, 'fit_intercept': True})
	Train R^2: 0.841, Val  R^2: 0.845
	Train MSE: 5268.636, Val MSE: 5183.663
	Train MAE: 39.706, Val MAE: 38.561
[LinearRegression PF] Train R^2: 0.942, Val  R^2: 0.923
[LinearRegression PF] Train MSE: 1926.635, Val MSE: 2570.585
[LinearRegression PF] Train MAE: 22.472, Val MAE: 24.583
Ridge({'alpha': 1.0, 'fit_intercept': False})
	Train R^2: 0.902, Val  R^2: 0.896
	Train MSE: 3259.995, Val MSE: 3488.906
	Train MAE: 29.904, Val MAE: 30.981


In [28]:
""" [ASSERTS] Checks for task 3a. """
assert var_exists('all_x_pf') and all_x_pf.shape == (17379, 679)
assert var_exists('train_x_pf') and var_exists('train_y_pf') and train_x_pf.shape[0] == train_y_pf.shape[0]
assert var_exists('val_x_pf') and var_exists('val_y_pf') and val_x_pf.shape[0] == val_y_pf.shape[0]
assert var_exists('test_x_pf') and var_exists('test_y_pf') and test_x_pf.shape[0] == test_y_pf.shape[0]
assert train_x_pf.shape[0] == 13903 and val_x_pf.shape[0] == 1738 and test_x_pf.shape == val_x_pf.shape
assert train_x_pf.shape[1] == val_x_pf.shape[1]
assert np.amin(train_x_pf) >= 0.0 and np.amax(train_x_pf) <= 1.0

assert var_exists('ridge_nopf') and var_exists('lr_pf') and var_exists('ridge_pf')
assert ridge_nopf.coef_.shape == (14,)


### [Task 3b] (5 points) For each of the three models, print the three most important features (coef values and name). You can use 'get_feature_names_out'.

In [29]:
"""Put your code here (~5-7 lines).
"""
# YOUR CODE HERE
feat_names_nopf = np.array(features)
feat_names_pf = polyf.get_feature_names_out(features)

def print_top3(model, names, label):
    top_idx = np.argsort(np.abs(model.coef_))[-3:][::-1]
    print(f"\n{label} top 3 features:")
    for i in top_idx:
        print(f"{names[i]}: coef={model.coef_[i]:.6f}")

print_top3(ridge_nopf, feat_names_nopf, "ridge_nopf")
print_top3(lr_pf, feat_names_pf, "lr_pf")
print_top3(ridge_pf, feat_names_pf, "ridge_pf")


ridge_nopf top 3 features:
registered: coef=919.842940
hour: coef=53.907427
nsqrtc: coef=-36.520766

lr_pf top 3 features:
registered nsqrtc: coef=29427076.441264
season nsqrtc: coef=-17279608.703701
registered nsqrtc^2: coef=-15725028.916407

ridge_pf top 3 features:
registered^3: coef=-781.171044
hour^3: coef=-727.502021
hour registered^2: coef=561.149185


In [30]:
""" THIS CODE CELL IS INTENTIONALLY LEFT EMPTY.  Remove the 'raise NotImplementedError' line. You should leave this cell empty (it is used for autograding). (Do not change the cell type from code to markdown.)"""
# THIS CODE CELL IS INTENTIONALLY LEFT EMPTY --- DO NOT MODIFY THIS CELL
# YOUR CODE HERE

" THIS CODE CELL IS INTENTIONALLY LEFT EMPTY.  Remove the 'raise NotImplementedError' line. You should leave this cell empty (it is used for autograding). (Do not change the cell type from code to markdown.)"

### [Task 3c] (5 points) Out of the three models, which would you use and why?

In [31]:
""" Remove the 'raise NotImplementedError' line. Write your answer as a comment in the place provided.  (Do not change the cell type from code to markdown.)"""
# 
## Answer: 
#
# YOUR CODE HERE
# I would choose lr_pf(Linear Regression Model with Polynomial Features)
# It has the best validation metrics among the three models:
# Val R^2 = 0.923 (highest), Val MSE = 2570.585 (lowest), and Val MAE = 24.583 (lowest).


" Remove the 'raise NotImplementedError' line. Write your answer as a comment in the place provided.  (Do not change the cell type from code to markdown.)"

## [Task 4] (25 points) Trees and Bagging Ensembles!

### Let's do some cleanup and sanity checking.

In [32]:
# let's do some cleanup and discard all the polynomial features stuff.
if var_exists('all_x_pf'):
    del all_x_pf, train_x_pf, train_y_pf, test_x_pf, test_y_pf, val_x_pf, val_y_pf

# sanity check shapes
train_x.shape, train_y.shape, test_x.shape, test_y.shape, val_x.shape, val_y.shape
assert train_x.shape == (13903, 14) and val_x.shape == (1738, 14) and test_x.shape == (1738, 14)

### [Task 4a] (5 points) Suppose a sklearn decision tree has n nodes. How many total splits does it contain? (Hint: think of it as a CS question not an ML question. Also think about edge cases (e.g., n=0).)

In [33]:
"""Put your code here to answer by implementing the function.
"""
def num_dt_splits(nodes): # should return the total number of splits in a decision tree. Must return an integer value.
    assert nodes >= 0
    # YOUR CODE HERE
    return max(0, nodes-1)

In [34]:
""" [ASSERTS] Check task 4a. """
assert var_exists('num_dt_splits')
assert num_dt_splits(0) == 0 and num_dt_splits(1) == 0
for i in range(0, 100):
    nodes = np.random.randint(0, 1e8)
    splits = num_dt_splits(nodes)
    assert splits>=0 and splits == int(splits)


### Let's train a decision tree!

### [Task 4b] (5 points) Train and evaluate two (regression) decision trees with sklearn's DecisionTreeRegressor. The first 'dtmodel' should be trained with default parameters (set the seed). The second 'dtregmodel' should be trained with the default parameters (set the seed) but a max depth of 7. Use r2_mse_mae_eval to print performance of both models.

In [35]:
"""Put your code here (~5 lines)
"""
# YOUR CODE HERE
from sklearn.tree import DecisionTreeRegressor

dtmodel = DecisionTreeRegressor(random_state=seed).fit(train_x, train_y)
_ = r2_mse_mae_eval(dtmodel, train_x, train_y, val_x, val_y, pref='[{}] '.format(dtmodel.__class__.__name__))

dtregmodel = DecisionTreeRegressor(max_depth=7,random_state=seed).fit(train_x, train_y)
_ = r2_mse_mae_eval(dtregmodel, train_x, train_y, val_x, val_y, pref='[{} max_depth=7] '.format(dtregmodel.__class__.__name__))

[DecisionTreeRegressor] Train R^2: 1.000, Val  R^2: 0.970
[DecisionTreeRegressor] Train MSE: 0.000, Val MSE: 1014.636
[DecisionTreeRegressor] Train MAE: 0.000, Val MAE: 10.080
[DecisionTreeRegressor max_depth=7] Train R^2: 0.955, Val  R^2: 0.957
[DecisionTreeRegressor max_depth=7] Train MSE: 1508.853, Val MSE: 1451.577
[DecisionTreeRegressor max_depth=7] Train MAE: 18.368, Val MAE: 18.929


In [36]:
""" [ASSERTS] Check task 4b. """
assert var_exists('dtmodel') and var_exists('dtregmodel') 
assert dtmodel.tree_.max_depth >= 6 and dtregmodel.tree_.max_depth == 7


### [Task 4c] (5 points) Is the regularized model overfitted? Is it better than the models trained in Tasks 2 and 3? (A few sentences suffice.)

In [37]:
""" Remove the 'raise NotImplementedError' line. Write your answer as a comment in the place provided.  (Do not change the cell type from code to markdown.)"""
# 
## Answer: 
#
# YOUR CODE HERE
# The regularized tree (max_depth=7) does not appear overfitted because its train and validation R^2 are very close
# (train ≈ 0.955 vs val ≈ 0.957), unlike the unregularized tree which achieves perfect train R^2 = 1.0.
# It is also better than the models from Tasks 2 and 3, since its validation R^2 (~0.957) is higher than the
# linear models in Task 2 (~0.845) and the polynomial/regularized linear models in Task 3 (~0.923 and ~0.896).


" Remove the 'raise NotImplementedError' line. Write your answer as a comment in the place provided.  (Do not change the cell type from code to markdown.)"

### Let's explore bagging.

### [Task 4d] (5 points) Train two ensemble models using sklearn's BaggingRegressor. Use default parameters but be sure to set the seed and set max_samples=0.5. The first ensemble 'lr_bagging' must use 'LinearRegression' as weak learners, whereas the second 'dtr_bagging' must use 'DecisionTreeRegressor' as weak learners.

In [38]:
"""Put your code here (~3 lines)
"""
# YOUR CODE HERE
from sklearn.ensemble import BaggingRegressor

lr_bagging = BaggingRegressor(estimator=LinearRegression(), max_samples=0.5, random_state=seed).fit(train_x, train_y)
dtr_bagging = BaggingRegressor(estimator=DecisionTreeRegressor(), max_samples=0.5, random_state=seed).fit(train_x, train_y)

_ = r2_mse_mae_eval(lr_bagging, train_x, train_y, val_x, val_y, pref='[Linear Regression Bagging] ')
print()
_ = r2_mse_mae_eval(dtr_bagging, train_x, train_y, val_x, val_y, pref='[DT Regression Bagging aka Random Forest] ')

[Linear Regression Bagging] Train R^2: 0.841, Val  R^2: 0.845
[Linear Regression Bagging] Train MSE: 5271.255, Val MSE: 5185.668
[Linear Regression Bagging] Train MAE: 39.617, Val MAE: 38.498

[DT Regression Bagging aka Random Forest] Train R^2: 0.989, Val  R^2: 0.982
[DT Regression Bagging aka Random Forest] Train MSE: 378.306, Val MSE: 609.102
[DT Regression Bagging aka Random Forest] Train MAE: 5.898, Val MAE: 8.416


In [39]:
""" [ASSERTS] Check task 4d. """
assert var_exists('lr_bagging') and var_exists('dtr_bagging') 
assert lr_bagging.random_state == dtr_bagging.random_state and dtr_bagging.random_state == seed


### [Task 4e] (5 points) Answer the following questions: 
### 1. Is the linear regression bagging ensemble a better model than linear regression (Task 2)? Is this expected? (We are looking for an explanation of why this approach performs the way it does.)
### 2. How does the random forest compare to all other models in this and previous task?

In [40]:
""" Remove the 'raise NotImplementedError' line. Write your answer as a comment in the place provided.  (Do not change the cell type from code to markdown.)"""
# 
## Answer: 
#
# YOUR CODE HERE
# 1. As seen in the experiments, the metrics are essentially the same for the bagged linear regression and the single linear regression, 
# which suggests that bagging did not provide any significant improvement for the linear model. This is likely because 
# linear regression is a low-variance model, so averaging multiple linear regressions does not reduce variance much.

# 2. The random-forest-style bagging model is the best overall in my runs:
#    Val R^2 = 0.982 (highest), Val MSE = 609.102 (lowest), Val MAE = 8.416 (lowest),
#    which is much better than Task 2 linear models, Task 3 polynomial/ridge models, and single trees in Task 4.

" Remove the 'raise NotImplementedError' line. Write your answer as a comment in the place provided.  (Do not change the cell type from code to markdown.)"